In [34]:
from fastai.vision.all import *
import pandas as pd
import pydicom

# Define image loading function using pydicom
def load_dicom_image(study_id, series_id, instance_number):
    dicom_path = f'rsna-2024-lumbar-spine-degenerative-classification/train_images/{study_id}/{series_id}/{instance_number}.dcm'
    dicom = pydicom.dcmread(dicom_path)
    img = dicom.pixel_array.astype(np.float32)
    img = (img - np.min(img)) / (np.max(img) - np.min(img))  # Normalize the pixel values
    img = img.astype(np.uint8)  # Convert to 8-bit integer
    return PILImage.create(img)
    # return img
    # Convert to a tensor that FastAI can work with
    # return TensorImage(img[None])  # Adding the channel axis

# Load the CSV file
df = pd.read_csv('rsna-2024-lumbar-spine-degenerative-classification/train_label_coordinates.csv')

# Custom function to get labels and coordinates
def get_labels(row):
    condition = row['condition']
    return [condition]

def get_coordinates(row):
    return (float(row[6]), float(row[6])) # get x, y coordinates

def get_x(row):
    return load_dicom_image(row['study_id'], row['series_id'], row['instance_number'])

# Create a Pipeline for get_y
get_y_pipeline = Pipeline([get_labels, get_coordinates])

# Load MRI images and annotations
# path = Path('rsna-2024-lumbar-spine-degenerative-classification/training/')
# df['image_path'] = df.apply(lambda row: path/f"{row['study_id']}_{row['series_id']}_{row['instance_number']}.jpg", axis=1)

# Function to combine condition labels and coordinates into a tuple
def get_y(row):
    condition = row['condition']
    coordinates = (row['x'], row['y'])
    return [condition], coordinates  # Multi-label output (condition, coordinates)

# Define a pipeline for both condition and coordinates
# class ConditionAndCoordinates(Transform):
#     def encodes(self, row):
#         condition = row['condition']
#         coordinates = (float(row['x']), float(row['y']))
#         return [condition], coordinates  # Return multi-label (condition, coordinates)
    
#     def decodes(self, o):
#         return o

def get_y(row):
    condition = row['condition']
    coordinates = (row['x'], row['y'])
    return [condition], coordinates  # Multi-label output (condition, coordinates)

# Use the transform pipeline for both the condition and the coordinates
condition_coordinates_pipeline = Pipeline([ConditionAndCoordinates()])

import sys
sys.setrecursionlimit(10000)

# FastAI DataBlock
# DataBlock for image classification and coordinates
spine_block = DataBlock(
    blocks=(ImageBlock(cls=PILImageBW), (MultiCategoryBlock, PointBlock)),  # Image and multi-output (labels and points)
    get_x=lambda row: load_dicom_image(row['study_id'], row['series_id'], row['instance_number']),  # Load the DICOM image
    # get_y=get_y_pipeline,  # Condition labels and coordinates
    get_y=get_y,
    splitter=RandomSplitter(valid_pct=0.2),  # Train/validation split
    item_tfms=Resize(256),  # Resize images
    batch_tfms=aug_transforms(size=224)  # Data augmentations
)
dls = spine_block.dataloaders(df, bs=32)

# Show a batch of images and labels
dls.show_batch(max_n=6, figsize=(10,7))


RecursionError: maximum recursion depth exceeded

In [35]:
# Custom function for labels and coordinates
def get_labels(row):
    condition = row['condition']
    return [condition]

def get_coordinates(row):
    return (float(row['x']), float(row['y']))  # get x, y coordinates

get_y_pipeline = Pipeline([get_labels, get_coordinates])

# Test pipeline
df_test = df.head()  # Test with a small sample
for _, row in df_test.iterrows():
    print(get_y_pipeline(row))

TypeError: list indices must be integers or slices, not str

In [36]:
from fastai.data.transforms import Pipeline

# Define the pipeline
get_y_pipeline = Pipeline([get_labels, get_coordinates])

# Test with a small DataFrame
df_test = df.head()  # Test with a small sample

# Apply the pipeline
for _, row in df_test.iterrows():
    labels = get_labels(row)
    coordinates = get_coordinates(row)
    print("Labels:", labels)
    print("Coordinates:", coordinates)
    print("Pipeline output:", get_y_pipeline(row))


Labels: ['Spinal Canal Stenosis']
Coordinates: (322.83185840707966, 227.9646017699115)


TypeError: list indices must be integers or slices, not str